In [6]:
import json
from pathlib import Path

import keras
from keras import Sequential, Input
import keras_tuner
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Dropout, Dense
from keras.metrics import RootMeanSquaredError
from keras.src.optimizers import Adam

from utils.centralized_training import _build_centralized_model_dataset
from gossiplearning.config import parse_config
from keras.src.layers import RepeatVector

In [7]:

with open("config.json", "r") as f:
    config = json.load(f)
    config = parse_config(config)

In [8]:
def model_builder(hp: keras_tuner.HyperParameters) -> keras.Model:
    model = Sequential()

    model.add(
        LSTM(
            hp.Int(min_value=16, max_value=320, step=16, name="lstm_layer_1_dim"),
            activation="tanh",
            input_shape=(16, 4),
        )
    )

    if hp.Boolean("dropout_1"):
        model.add(
            Dropout(hp.Float(name="dropout_1_rate", min_value=0.1, max_value=0.5, step=0.05))
        )

    if hp.Boolean("lstm_2"):
        model.add(RepeatVector(1)),
        model.add(
            LSTM(
                hp.Int(min_value=16, max_value=320, step=16, name="lstm_layer_2_dim"),
                activation="tanh",
                return_sequences=True,
            )
        )

    if hp.Boolean("dropout_2"):
        model.add(
            Dropout(hp.Float(name="dropout_2_rate", min_value=0.1, max_value=0.5, step=0.05))
        )

    if hp.Boolean("dense"):
        model.add(
            Dense(
                hp.Int(min_value=16, max_value=320, step=16, name="dense_dim"),
                activation="relu"
            )
        )

    model.add(
        Dense(4, activation='relu')
    )

    optz = Adam(
        learning_rate=hp.Float(min_value=0.0005, max_value=0.01, step=0.0005, name="learning_rate")
    )

    model.compile(optimizer=optz, loss='mse', metrics=["mae", "msle", "mse", "mape", RootMeanSquaredError()])
    return model

In [9]:
tuner = keras_tuner.RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=30,
)

2023-07-10 08:49:17.149979: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-07-10 08:49:17.150008: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-07-10 08:49:17.150018: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-07-10 08:49:17.150052: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-10 08:49:17.150072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
train, val, _ = _build_centralized_model_dataset(Path("data/datasets/4func_10nodes/0"), config)

In [11]:
early_stopping = EarlyStopping(
        monitor="val_loss",
        patience=config.training.patience,
        min_delta=config.training.min_delta,
        restore_best_weights=True
    )

tuner.search(train[0], train[1], validation_data=val, callbacks=[early_stopping], epochs=1000)

Trial 30 Complete [00h 05m 12s]
val_loss: 0.02101859264075756

Best val_loss So Far: 0.020821115002036095
Total elapsed time: 01h 41m 54s
INFO:tensorflow:Oracle triggered exit


In [12]:
tuner.results_summary(num_trials=1)

Results summary
Results in ./untitled_project
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 07 summary
Hyperparameters:
lstm_layer_1_dim: 48
dropout_1: True
lstm_2: True
dropout_2: False
dense: True
learning_rate: 0.006
dense_dim: 224
dropout_1_rate: 0.1
dropout_2_rate: 0.2
lstm_layer_2_dim: 112
Score: 0.020821115002036095
